<a href="https://colab.research.google.com/github/raviakasapu/LLM-Training-Docs/blob/main/03_fine_tune_sentence_transformers_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine-tune sentence transformers for classification

* documentation --> https://aws.amazon.com/blogs/machine-learning/create-and-fine-tune-sentence-transformers-for-enhanced-classification-accuracy/

* dataset --> https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020?resource=download

In [1]:
!pip install -q sentencepiece
!pip install -q sentence_transformers
!pip install -q xgboost
!pip install -q scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00


In [2]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00


In [3]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
from datasets import load_dataset


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("promptcloud/amazon-product-dataset-2020")

print("Path to dataset files:", path)

100%|██████████| 5.37M/5.37M [00:00<00:00, 84.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/promptcloud/amazon-product-dataset-2020/versions/1


In [6]:
import os
import pandas as pd
# Find the CSV file in the downloaded path
csv_file = [f for f in os.listdir(path)][0]
file_path = os.path.join(path, csv_file)


In [7]:
#!/bin/bash
!curl -L -o amazon-product-dataset-2020.zip\
  https://www.kaggle.com/api/v1/datasets/download/promptcloud/amazon-product-dataset-2020

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5495k  100 5495k    0     0   9.8M      0 --:--:-- --:--:-- --:--:-- 37.5M


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import zipfile

# Read the CSV file directly from the zip file
with zipfile.ZipFile('/content/amazon-product-dataset-2020.zip') as z:
    # Get the CSV filename from the zip
    csv_filename = [f for f in z.namelist() if f.endswith('.csv')][0]

    # Read the CSV using pandas
    data = pd.read_csv(z.open(csv_filename))

data.columns = data.columns.str.lower().str.replace(' ', '_')
data['main_category'] = data['category'].str.split("|").str[0]
data["all_text"] = data.apply(
    lambda r: " ".join(
        [
            str(r["product_name"]) if pd.notnull(r["product_name"]) else "",
            str(r["about_product"]) if pd.notnull(r["about_product"]) else "",
            str(r["product_specification"]) if pd.notnull(r["product_specification"]) else "",
            str(r["technical_details"]) if pd.notnull(r["technical_details"]) else ""
        ]
    ),
    axis=1
)
label_encoder = LabelEncoder()
labels_transform = label_encoder.fit_transform(data['main_category'])
data['label']=labels_transform
data[['all_text','label']]

,all_text,label
0,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",19
1,"Electronic Snap Circuits Mini Kits Classpack, ...",21
2,3Doodler Create Flexy 3D Printing Filament Ref...,21
3,Guillow Airplane Design Studio with Travel Cas...,21
4,Woodstock- Collage 500 pc Puzzle Make sure thi...,21
...,...,...
9997,Remedia Publications REM536B Money Activity Bo...,21
9998,Trends International NFL La Chargers HG - Mobi...,21
9999,NewPath Learning 10 Piece Science Owls and Owl...,14
10000,Disney Princess Do It Yourself Braid Set Prod...,21


In [9]:
data.columns

Index(['uniq_id', 'product_name', 'brand_name', 'asin', 'category',
       'upc_ean_code', 'list_price', 'selling_price', 'quantity',
       'model_number', 'about_product', 'product_specification',
       'technical_details', 'shipping_weight', 'product_dimensions', 'image',
       'variants', 'sku', 'product_url', 'stock', 'product_details',
       'dimensions', 'color', 'ingredients', 'direction_to_use',
       'is_amazon_seller', 'size_quantity_variant', 'product_description',
       'main_category', 'all_text', 'label'],
      dtype='object')

In [10]:
print("\nDistribution of main categories for label 0:")
print(data[data['label'] == 23]['main_category'].value_counts())


Distribution of main categories for label 0:
Series([], Name: count, dtype: int64)


In [11]:
data.to_csv('data.csv')

In [12]:
dataset = load_dataset('csv', data_files='data.csv', split='train')
#dataset = df.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
dataset_new = dataset.select_columns({'about_product','label'})
dataset_new = dataset_new.rename_column("about_product", "text")

In [14]:
filtered_dataset = dataset_new.filter(lambda x: x['label'] != 23)

Filter:   0%|          | 0/10002 [00:00<?, ? examples/s]

In [15]:
print(filtered_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 9172
})


In [16]:
print(filtered_dataset[0])

{'text': "Make sure this fits by entering your model number. | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. | COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. | INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest | BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family | BEYOND COMPARE: Try our skate

In [17]:
from transformers import AutoTokenizer

In [18]:
#train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

In [19]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
def preprocess_function(example):  # Note: single example, not examples
    # Ensure we have a string
    text = str(example["text"])

    # Tokenize
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Add label
    # Remove the extra batch dimension and ensure correct shape
    tokenized = {
        "input_ids": tokenized["input_ids"].squeeze(0),
        "attention_mask": tokenized["attention_mask"].squeeze(0),
        "labels": example["label"]
    }


    return tokenized

In [21]:
tokenized_df = filtered_dataset.map(
    preprocess_function,
    batched=False,  # Process one at a time
    remove_columns=dataset_new.column_names
)

Map:   0%|          | 0/9172 [00:00<?, ? examples/s]

In [22]:
tokenized_df_split = tokenized_df.train_test_split(test_size=0.1)

In [23]:
tokenized_df_split['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8254
})

In [24]:
print(filtered_dataset.column_names)

['text', 'label']


In [25]:
print(filtered_dataset["text"][:2])  # print first two examples

["Make sure this fits by entering your model number. | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern that’s second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for people 120 to 170 pounds. | COREFLEX TECH: CoreFlex construction is water resistant, impact resistant, scratch resistant and has a flex like you won’t believe. These boards combine fiberglass, epoxy, HD plastic and bamboo to create a perfect blend of performance and strength. | INSPIRED BY THE NORTHWEST: Our founding ideal is chasing adventure & riding the best boards possible, inspired by the hills, waves, beaches & mountains all around our headquarters in the Northwest | BEST IN THE WORLD: DB was founded out of sheer love of longboarding with a mission to create the best custom longboards in the world, to do it sustainably, & to treat customers & employees like family | BEYOND COMPARE: Try our skateboards &

In [26]:
print(type(dataset_new))

<class 'datasets.arrow_dataset.Dataset'>


In [27]:
# First convert categories to string and handle any NaN values
unique_pairs = data[['main_category', 'label']].dropna().drop_duplicates()
unique_pairs['main_category'] = unique_pairs['main_category'].astype(str)
unique_pairs['label'] = unique_pairs['label'].astype(str)

# Sort after conversion
unique_pairs = unique_pairs.sort_values('main_category')
unique_pairs['label'] = unique_pairs['label'].astype(int)


# Create the mappings
id2label = dict(zip(unique_pairs['main_category'], unique_pairs['label']))
label2id = dict(zip(unique_pairs['label'], unique_pairs['main_category']))

# Sort the dictionaries
id2label = dict(sorted(id2label.items()))
label2id = dict(sorted(label2id.items()))


# Print the mappings to verify
print("id2label:", id2label)
print("label2id:", label2id)

id2label: {'Arts, Crafts & Sewing ': 0, 'Automotive ': 1, 'Baby Products ': 2, 'Beauty & Personal Care ': 3, 'Cell Phones & Accessories ': 4, 'Clothing, Shoes & Jewelry ': 5, 'Electronics ': 6, 'Grocery & Gourmet Food ': 7, 'Health & Household ': 8, 'Hobbies ': 9, 'Home & Kitchen ': 10, 'Industrial & Scientific ': 11, 'Movies & TV ': 12, 'Musical Instruments ': 13, 'Office Products ': 14, 'Patio, Lawn & Garden ': 15, 'Pet Supplies ': 16, 'Remote & App Controlled Vehicle Parts ': 17, 'Remote & App Controlled Vehicles & Parts ': 18, 'Sports & Outdoors ': 19, 'Tools & Home Improvement ': 20, 'Toys & Games ': 21, 'Video Games ': 22}
label2id: {0: 'Arts, Crafts & Sewing ', 1: 'Automotive ', 2: 'Baby Products ', 3: 'Beauty & Personal Care ', 4: 'Cell Phones & Accessories ', 5: 'Clothing, Shoes & Jewelry ', 6: 'Electronics ', 7: 'Grocery & Gourmet Food ', 8: 'Health & Household ', 9: 'Hobbies ', 10: 'Home & Kitchen ', 11: 'Industrial & Scientific ', 12: 'Movies & TV ', 13: 'Musical Instrument

In [28]:
print(len(label2id))
print(len(id2label))

23
23


In [29]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=23, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args = TrainingArguments(
    output_dir="classification_distilbert-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


In [31]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [32]:
import evaluate

accuracy = evaluate.load("accuracy")

In [33]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [34]:
import tensorflow as tf
tf.experimental.numpy.experimental_enable_numpy_behavior()


In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df_split['train'],
    eval_dataset=tokenized_df_split['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raviakasapu to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
